In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import docker
import time
import concurrent.futures
from datetime import datetime
import copy
import math
from collections import defaultdict
import pprint
from collections import namedtuple
from datetime import datetime
import os
import ctypes as ct
import multiprocessing
import shutil

In [2]:
# Discover the hardware architecture.
avail_cores = os.cpu_count()
print(f"Available cores: {avail_cores}")

Available cores: 48


### Core-level co-location experiments

In [4]:
def parse_start_time(start_time_str):
    if '.' in start_time_str:
        time_part, rest = start_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(start_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def parse_die_time(die_time_str):
    if '.' in die_time_str:
        time_part, rest = die_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(die_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def run_container(task):
    label = task.get("label")
    run_kwargs = {}
    if "environment" in task:
        run_kwargs["environment"] = task["environment"]
    if "volumes" in task:
        run_kwargs["volumes"] = task["volumes"]
    if "working_dir" in task:
        run_kwargs["working_dir"] = task["working_dir"]
    container = task['client'].containers.run(
        image=task['image'],
        command=task['command'],
        cpuset_cpus=task.get('cpuset_cpus'),
        cgroupns="private",
        detach=True,
        labels = {"test": label} if label else {},
        **run_kwargs
    )
    container.wait()
    container.reload()
    status = container.attrs['State']['ExitCode']
    if status != 0:
        print(container.logs().decode())
    die_time = parse_die_time(container.attrs['State']['FinishedAt'])
    started_at = container.attrs['State']['StartedAt']
    start_time = parse_start_time(started_at)
    container_lifetime = (die_time - start_time).total_seconds()
    print(f"Container lifetime: {container_lifetime} seconds")
    print(f"Container on CPU {task.get('cpuset_cpus')} completed and removed.")
    isolated_benchmarking_results[container.name] = {
        # 'workload': container.attrs['Config']['Labels'],
        'coloc_pair': None,
        'workload': container.attrs['Config']['Labels'].get('test', None), 
        'id': container.id,
        'life_time': container_lifetime,
    }
    container.remove()

if __name__ == "__main__":
    client = docker.from_env()
    isolated_benchmarking_results = {}

    # Prepare disk files (block until done)
    print("Container started for Disk benchmark (prepare).")
    client.containers.run(
        image="niklas/sysbench:latest",
        command=[
            "sysbench", "fileio",
            "--file-total-size=50G",
            "--file-num=128",
            "--file-block-size=4K",
            "--file-extra-flags=direct",
            "prepare"
        ],
        volumes={"/storage/sysbench_data": {"bind": "/data", "mode": "rw"}},
        working_dir="/data",
        detach=False,
        auto_remove=True,
        cpuset_cpus="4",
        cgroupns="private"
    )

    tasks = [
        {"client": client, "image": "linpack:latest", "command": ["linpack"], "cpuset_cpus": "0", "label": "cpu", "environment": {"LINPACK_ARRAY_SIZE": "1000"}},

        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "memory", "run",
            "--memory-total-size=300G",
            "--memory-block-size=4K",
             "--time=0"
        ], "cpuset_cpus": "2","label": "mem",},

        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "fileio",
            "--file-total-size=50G",
            "--file-num=128",
            "--file-test-mode=seqwr",
            "--file-block-size=4K",
            "--file-extra-flags=direct",
            "--events=13107200",   # 50*1024*1024*1024/4096
            "--time=0",
            "run"
        ], "cpuset_cpus": "4", "label": "fileio","volumes": {"/storage/sysbench_data": {"bind": "/data", "mode": "rw"}}, "working_dir": "/data"}
    ]

    with concurrent.futures.ThreadPoolExecutor(max_workers=3, thread_name_prefix="Isolator") as executor:
        futures = [executor.submit(run_container, task) for task in tasks]
    for future in futures:
        future.result()

    # Disk cleanup (block until done)
    print("Disk cleanup...")
    client.containers.run(
        image="niklas/sysbench:latest",
        command=[
            "sysbench", "fileio",
            "--file-total-size=200G",
            "--file-num=128",
            "--file-block-size=16K",
            "--file-extra-flags=direct",
            "cleanup"
        ],
        volumes={"/storage/sysbench_data": {"bind": "/data", "mode": "rw"}},
        working_dir="/data",
        detach=False,
        auto_remove=True,
        cpuset_cpus="4",
        cgroupns="private"
    )
    print("Disk benchmark completed.")
    print("All isolated benchmarks completed.")
    client.close()

Container started for Disk benchmark (prepare).
Container lifetime: 15.550664 seconds
Container on CPU 2 completed and removed.
Memory required:  7824K.


LINPACK benchmark, Double precision.
Machine precision:  15 digits.
Array size 1000 X 1000.
Average rolled and unrolled performance:

    Reps Time(s) DGEFA   DGESL  OVERHEAD    KFLOPS
----------------------------------------------------
      32   0.61  89.96%   0.76%   9.28%  9618895.488
      64   1.23  89.95%   0.75%   9.29%  9636318.191
     128   2.45  89.94%   0.75%   9.31%  9662483.902
     256   4.94  90.00%   0.75%   9.25%  9583348.218
     512   9.89  90.01%   0.75%   9.23%  9564662.565
    1024  19.62  89.95%   0.75%   9.30%  9647287.713


Container lifetime: 39.575586 seconds
Container on CPU 0 completed and removed.
Container lifetime: 182.115732 seconds
Container on CPU 4 completed and removed.
Disk cleanup...
Disk benchmark completed.
All isolated benchmarks completed.


In [ ]:
# stress-ng &f fio
def parse_start_time(start_time_str):
    if '.' in start_time_str:
        time_part, rest = start_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(start_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def parse_die_time(die_time_str):
    if '.' in die_time_str:
        time_part, rest = die_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(die_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def run_container(task):
    label = task.get("label")
    run_kwargs = {}
    if "environment" in task:
        run_kwargs["environment"] = task["environment"]
    if "volumes" in task:
        run_kwargs["volumes"] = task["volumes"]
    if "working_dir" in task:
        run_kwargs["working_dir"] = task["working_dir"]
    container = task['client'].containers.run(
        image=task['image'],
        command=task['command'],
        cpuset_cpus=task.get('cpuset_cpus'),
        cgroupns="private",
        detach=True,
        labels = {"test": label} if label else {},
        **run_kwargs
    )
    container.wait()
    container.reload()
    status = container.attrs['State']['ExitCode']
    if status != 0:
        print(container.logs().decode())
    die_time = parse_die_time(container.attrs['State']['FinishedAt'])
    started_at = container.attrs['State']['StartedAt']
    start_time = parse_start_time(started_at)
    container_lifetime = (die_time - start_time).total_seconds()
    print(f"Container lifetime: {container_lifetime} seconds")
    print(f"Container on CPU {task.get('cpuset_cpus')} completed and removed.")
    isolated_benchmarking_results[container.name] = {
        # 'workload': container.attrs['Config']['Labels'],
        'coloc_pair': None,
        'workload': container.attrs['Config']['Labels'].get('test', None), 
        'id': container.id,
        'life_time': container_lifetime,
    }
    container.remove()

if __name__ == "__main__":
    client = docker.from_env()
    isolated_benchmarking_results = {}

    tasks = [
        # {"client": client, "image": "linpack:latest", "command": ["linpack"], "cpuset_cpus": "0", "label": "cpu", "environment": {"LINPACK_ARRAY_SIZE": "1000"}},
        {"client": client, "image": "stress-ng-cpu:latest", "command": ["stress-ng", "--cpu", "1", "--cpu-method", "matrixprod", "--cpu-ops", "100000", "--metrics-brief"], "cpuset_cpus": "0", "label": "cpu"},


        # With vm allocation
        {"client": client, "image": "stress-ng-mem-vm:latest", "command": ["stress-ng", "--vm", "1", "--vm-bytes", "18G", "--vm-ops", "1000", "--metrics-brief"], "cpuset_cpus": "2", "label": "mem"},

        # With stream allocation
        # {"client": client, "image": "stress-ng-mem-stream:latest", "command": ["stress-ng", "--stream", "4", "--stream-ops", "5000", "--metrics-brief"], "cpuset_cpus": "2", "label": "mem"},

        {"client": client, "image": "fio:latest", "command": ["fio", "--name", "seqread", "--rw", "read", "--bs", "1M", "--size", "18G", "--numjobs", "1", "--readonly","1","--direct","1","iodepth","32","--ioengine","io_uring","--group_reporting"], "cpuset_cpus": "4", "label": "fileio"}
    ]

    # with concurrent.futures.ThreadPoolExecutor(max_workers=3, thread_name_prefix="Isolator") as executor:
    #     futures = [executor.submit(run_container, task) for task in tasks]
    # for future in futures:
    #     future.result()

    for task in tasks:
        run_container(task)

    print("All isolated benchmarks completed.")
    client.close()

KeyboardInterrupt: 

In [5]:
for name, container_id in isolated_benchmarking_results.items():
    print(f"{name}: {container_id}")

infallible_pascal: {'coloc_pair': None, 'workload': 'mem', 'id': '916d0d9ebdc275afe56f723beddd91601fc8b33a0c16ba7f672233b853b1cb9a', 'life_time': 0.86674}
inspiring_aryabhata: {'coloc_pair': None, 'workload': 'cpu', 'id': '4e1ee68282f1828e8ddf235bd402c22debe55fbe137635cb34cd6e9eb824ef53', 'life_time': 10.564683}
xenodochial_euler: {'coloc_pair': None, 'workload': 'fileio', 'id': '049d926d6e44d55644d3aaa0831787bc30c1fd257d1408acbf7f89c5ee2dbc86', 'life_time': 14.073507}


In [9]:
# Access the watched benchmark containers for runtime and power consumption.
# Add power consumption caputured by ebpf-mon.
output_dir = "output"

print("Container names and IDs:")
for name, container_id in isolated_benchmarking_results.items():
    print(f"{name}: {container_id}")

# Collect container IDs
container_ids = set(info['id'] for info in isolated_benchmarking_results.values())

for entry in os.listdir(output_dir):
    entry_path = os.path.join(output_dir, entry)
    if os.path.isdir(entry_path):
        # If entry is NOT part of any container ID, remove the whole directory and continue
        if not any(entry in cid for cid in container_ids):
            # print(f"Removing directory: {entry_path}")
            shutil.rmtree(entry_path)
            continue  # Don't process contents of a deleted directory

        # If entry IS part of a container ID, remove all subfolders/files not starting with "power"
        for metric_entry in os.listdir(entry_path):
            metric_entry_path = os.path.join(entry_path, metric_entry)
            if not metric_entry.startswith("power"):
                if os.path.isdir(metric_entry_path):
                    # print(f"Removing directory: {metric_entry_path}")
                    shutil.rmtree(metric_entry_path)
                else:
                    # print(f"Removing file: {metric_entry_path}")
                    os.remove(metric_entry_path)
            else:
                # If the entry is a power metric, keep it
                print(f"Keeping power metric: {metric_entry_path}")
                for file in os.listdir(metric_entry_path):
                    file_path = os.path.join(metric_entry_path, file)
                    print(f"Keeping file: {file_path}")
                    df = pd.read_csv(file_path)
                    container_name = df['container_name'].iloc[0] if 'container_name' in df.columns else None
                    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
                    df.set_index('timestamp', inplace=True)
                    power_mean = df['value'].mean()
                    print(f"Power mean for {file_path}: {power_mean}")
                    isolated_benchmarking_results[container_name]['power'] = float(power_mean)
# Print the final isolated benchmarking results with power consumption
print("Final isolated benchmarking results with power consumption:")
pprint.pprint(isolated_benchmarking_results)

Container names and IDs:
confident_germain: {'coloc_pair': None, 'workload': 'mem', 'id': 'a798aa37ddbbb4605b377e9c10c4109dc638986f94455a8c3c1e8b0af926d2e2', 'life_time': 15.550664}
angry_poitras: {'coloc_pair': None, 'workload': 'cpu', 'id': 'fb1815ced0235244b8ccdef023637e2e6f6f9220342ac53f0877de5c3507b39c', 'life_time': 39.575586}
musing_goldberg: {'coloc_pair': None, 'workload': 'fileio', 'id': '109184dbe77cc8afba6fb59ad39c6db7ffef2a92070487a9d84b649af0f4ef27', 'life_time': 182.115732}
Keeping power metric: output/a798aa37ddbb/power
Keeping file: output/a798aa37ddbb/power/timeseries.csv
Power mean for output/a798aa37ddbb/power/timeseries.csv: 0.15277761160110087
Keeping power metric: output/109184dbe77c/power
Keeping file: output/109184dbe77c/power/timeseries.csv
Power mean for output/109184dbe77c/power/timeseries.csv: 0.052417793183240254
Keeping power metric: output/fb1815ced023/power
Keeping file: output/fb1815ced023/power/timeseries.csv
Power mean for output/fb1815ced023/power/t

In [7]:
# Using linpack | Run co-located benchmarks in a Docker container on the same core physical core using logical pairs.
def parse_start_time(start_time_str):
    # Trim to microseconds and remove trailing 'Z'
    if '.' in start_time_str:
        time_part, rest = start_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(start_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def parse_die_time(die_time_str):
    if '.' in die_time_str:
        time_part, rest = die_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(die_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")
    
def run_container(task):
    label = task.get("label")
    run_kwargs = {}
    if "environment" in task:
        run_kwargs["environment"] = task["environment"]
    if "volumes" in task:
        run_kwargs["volumes"] = task["volumes"]
    if "working_dir" in task:
        run_kwargs["working_dir"] = task["working_dir"]
    container = task['client'].containers.run(
        image=task['image'],
        command=task['command'],
        cpuset_cpus=task.get('cpuset_cpus'),
        cgroupns="private",
        detach=True,
        labels = {"test": label} if label else {},
        **run_kwargs
    )
    
    container.wait()
    container.reload()
    status = container.attrs['State']['ExitCode']
    if status != 0:
        print(container.logs().decode())
    die_time = parse_die_time(container.attrs['State']['FinishedAt'])
    started_at = container.attrs['State']['StartedAt']
    start_time = parse_start_time(started_at)
    container_lifetime = (die_time - start_time).total_seconds()
    print(f"Container lifetime: {container_lifetime} seconds")
    coloc_benchmarking_results[container.name] = {
        'coloc_pair': pair_name,
        'workload': container.attrs['Config']['Labels'].get('test', None),  
        'id': container.id,
        'colocated_runtime': container_lifetime,
    }
    container.remove()
    print(f"Container on CPU {task.get('cpuset_cpus')} completed and removed.")
    
if __name__ == "__main__":
    
    # Initialize Docker client
    client = docker.from_env()
    
    # Prepare affinity score map for colocated pairs.
    coloc_benchmarking_results = {}

    # Init the disk benchmark.
    print("Container started for Disk benchmark (prepare).")
    client.containers.run(
        image="niklas/sysbench:latest",
        command=[
            "sysbench", "fileio",
            "--file-total-size=50G",
            "--file-num=128",
            "--file-block-size=4K",
            "--file-extra-flags=direct",
            "prepare"
        ],
        volumes={"/storage/sysbench_data": {"bind": "/data", "mode": "rw"}},
        working_dir="/data",
        detach=False,
        auto_remove=True,
        cpuset_cpus="4",
        cgroupns="private"
    )
    
    
    tasks = [
        {"client": client, "image": "linpack:latest", "command": ["linpack"], "cpuset_cpus": "0", "label": "cpu", "environment": {"LINPACK_ARRAY_SIZE": "1000"}},

        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "memory", "run",
            "--memory-total-size=300G",
            "--memory-block-size=4K",
            "--time=0"
        ], "cpuset_cpus": "2", "label": "mem"},

        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "fileio",
            "--file-total-size=50G",
            "--file-num=128",
            "--file-test-mode=seqwr",
            "--file-block-size=4K",
            "--file-extra-flags=direct",
            "--events=13107200",   # 50*1024*1024*1024/4096
            "--time=0",
            "run"
        ], "cpuset_cpus": "4", "label": "fileio","volumes": {"/storage/sysbench_data": {"bind": "/data", "mode": "rw"}}, "working_dir": "/data"}
    ]

    # Run every co-located benchmark combination on the same core.
    colocation = [
        {"CpuMem": [
        {"client": client, "image": "linpack:latest", "command": ["linpack"], "cpuset_cpus": "0", "label": "cpu", "environment": {"LINPACK_ARRAY_SIZE": "1000"}},
        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "memory", "run",
            "--memory-total-size=300G",
            "--memory-block-size=4K",
            "--time=0"
        ], "cpuset_cpus": "2", "label": "mem",},
        ]},
        {"MemFileIO": [
        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "memory", "run",
            "--memory-total-size=300G",
            "--memory-block-size=4K",
            "--time=0"
        ], "cpuset_cpus": "2", "label": "mem",},
        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "fileio",
            "--file-total-size=50G",
            "--file-num=128",
            "--file-test-mode=seqwr",
            "--file-block-size=4K",
            "--file-extra-flags=direct",
            "--events=13107200",   # 50*1024*1024*1024/4096
            "--time=0",
            "run"
        ], "cpuset_cpus": "4", "label": "fileio","volumes": {"/storage/sysbench_data": {"bind": "/data", "mode": "rw"}}, "working_dir": "/data"}
        ]},
        {"FileIOCpu": [
        # {"client": client, "image": "linpack:latest", "command": ["linpack"], "cpuset_cpus": "0", "label": "cpu", "environment": {"LINPACK_ARRAY_SIZE": "1000"}},
        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "fileio",
            "--file-total-size=50G",
            "--file-num=128",
            "--file-test-mode=seqwr",
            "--file-block-size=4K",
            "--file-extra-flags=direct",
            "--events=13107200",   # 50*1024*1024*1024/4096
            "--time=0",
            "run"
        ], "cpuset_cpus": "4", "label": "fileio","volumes": {"/storage/sysbench_data": {"bind": "/data", "mode": "rw"}}, "working_dir": "/data"},
        {"client": client, "image": "linpack:latest", "command": ["linpack"], "cpuset_cpus": "0", "label": "cpu", "environment": {"LINPACK_ARRAY_SIZE": "1000"}},]},
        {"CpuCpu": [
        {"client": client, "image": "linpack:latest", "command": ["linpack"], "cpuset_cpus": "0", "label": "cpu", "environment": {"LINPACK_ARRAY_SIZE": "1000"}},
        {"client": client, "image": "linpack:latest", "command": ["linpack"], "cpuset_cpus": "0", "label": "cpu", "environment": {"LINPACK_ARRAY_SIZE": "1000"}},
        ]},
        {"MemMem": [
        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "memory", "run",
            "--memory-total-size=300G",
            "--memory-block-size=4K",
            "--time=0"
        ], "cpuset_cpus": "2","label": "mem",},
        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "memory", "run",
            "--memory-total-size=300G",
            "--memory-block-size=4K",
            "--time=0"
        ], "cpuset_cpus": "2","label": "mem",},
        ]},
        {"FileIOFileIO": [
        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "fileio",
            "--file-total-size=50G",
            "--file-num=128",
            "--file-test-mode=seqwr",
            "--file-block-size=4K",
            "--file-extra-flags=direct",
            "--events=13107200",   # 50*1024*1024*1024/4096
            "--time=0",
            "run"
        ], "cpuset_cpus": "4", "label": "fileio","volumes": {"/storage/sysbench_data": {"bind": "/data", "mode": "rw"}}, "working_dir": "/data"},
        {"client": client, "image": "niklas/sysbench:latest", "command": [
            "sysbench", "fileio",
            "--file-total-size=50G",
            "--file-num=128",
            "--file-test-mode=seqwr",
            "--file-block-size=4K",
            "--file-extra-flags=direct",
            "--events=13107200",   # 50*1024*1024*1024/4096
            "--time=0",
            "run"
        ], "cpuset_cpus": "4", "label": "fileio","volumes": {"/storage/sysbench_data": {"bind": "/data", "mode": "rw"}}, "working_dir": "/data"}
        ]}
    ]
    
    for coloc in colocation:
            for pair_name, tasks in coloc.items():
                print(f"Running colocated tasks for: {pair_name}")
                with concurrent.futures.ThreadPoolExecutor(max_workers=3, thread_name_prefix="Colocator") as executor:
                    futures = [executor.submit(run_container, task) for task in tasks]
                for future in futures:
                    future.result()
                print(f"Completed colocated tasks for: {pair_name}")
    
    client.containers.run(
        image="niklas/sysbench:latest",
        command=[
            "sysbench", "fileio",
            "--file-total-size=200G",
            "--file-num=128",
            "--file-block-size=16K",
            "--file-extra-flags=direct",
            "cleanup"
        ],
        volumes={"/storage/sysbench_data": {"bind": "/data", "mode": "rw"}},
        working_dir="/data",
        detach=False,
        auto_remove=True,
        cpuset_cpus="4",
        cgroupns="private"
    )

    
    print("Disk benchmark completed.")
        
    print("All colocated benchmarks completed.")
    # Clean up Docker client
    client.close() 
    
print("Container names and IDs:")
for name, container_id in coloc_benchmarking_results.items():
    print(f"{name}: {container_id}")

Container started for Disk benchmark (prepare).
Running colocated tasks for: CpuMem
Container lifetime: 15.532565 seconds
Container on CPU 2 completed and removed.
Memory required:  7824K.


LINPACK benchmark, Double precision.
Machine precision:  15 digits.
Array size 1000 X 1000.
Average rolled and unrolled performance:

    Reps Time(s) DGEFA   DGESL  OVERHEAD    KFLOPS
----------------------------------------------------
      32   0.59  89.63%   0.72%   9.65%  10055745.163
      64   1.18  89.64%   0.72%   9.63%  10045888.666
     128   2.37  89.66%   0.72%   9.62%  10036417.555
     256   4.73  89.64%   0.72%   9.64%  10051454.693
     512   9.46  89.65%   0.72%   9.63%  10041630.093
    1024  18.93  89.65%   0.72%   9.63%  10035590.386


Container lifetime: 37.982324 seconds
Container on CPU 0 completed and removed.
Completed colocated tasks for: CpuMem
Running colocated tasks for: MemFileIO
Container lifetime: 15.584474 seconds
Container on CPU 2 completed and removed.
Containe

In [ ]:
# TODO: Put on NUMA-Cores and compare results, numa mapping done = (N, N+24) because I have 1 Socket, 24 physical cores, 1 NUMA Node, Each physical core has 2 logical cpus (SMT threads).
# Using stress-ng and fio
def parse_start_time(start_time_str):
    # Trim to microseconds and remove trailing 'Z'
    if '.' in start_time_str:
        time_part, rest = start_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(start_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def parse_die_time(die_time_str):
    if '.' in die_time_str:
        time_part, rest = die_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(die_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")
    
def run_container(task):
    label = task.get("label")
    run_kwargs = {}
    if "environment" in task:
        run_kwargs["environment"] = task["environment"]
    if "volumes" in task:
        run_kwargs["volumes"] = task["volumes"]
    if "working_dir" in task:
        run_kwargs["working_dir"] = task["working_dir"]
    container = task['client'].containers.run(
        image=task['image'],
        command=task['command'],
        cpuset_cpus=task.get('cpuset_cpus'),
        cgroupns="private",
        detach=True,
        labels = {"test": label} if label else {},
        **run_kwargs
    )
    
    container.wait()
    container.reload()
    status = container.attrs['State']['ExitCode']
    if status != 0:
        print(container.logs().decode())
    die_time = parse_die_time(container.attrs['State']['FinishedAt'])
    started_at = container.attrs['State']['StartedAt']
    start_time = parse_start_time(started_at)
    container_lifetime = (die_time - start_time).total_seconds()
    print(f"Container lifetime: {container_lifetime} seconds")
    coloc_benchmarking_results[container.name] = {
        'coloc_pair': pair_name,
        'workload': container.attrs['Config']['Labels'].get('test', None),  
        'id': container.id,
        'colocated_runtime': container_lifetime,
    }
    container.remove()
    print(f"Container on CPU {task.get('cpuset_cpus')} completed and removed.")
    
if __name__ == "__main__":
    
    # Initialize Docker client
    client = docker.from_env()
    
    # Prepare affinity score map for colocated pairs.
    coloc_benchmarking_results = {}

    # tasks = [
    #     {"client": client, "image": "stress-ng-cpu:latest", "command": ["stress-ng", "--cpu", "1", "--cpu-method", "matrixprod", "--cpu-ops", "100000", "--metrics-brief"], "cpuset_cpus": "0", "label": "cpu"},

    #     {"client": client, "image": "stress-ng-mem-vm:latest", "command": ["stress-ng", "--vm", "1", "--vm-bytes", "18G", "--vm-ops", "1000", "--metrics-brief"], "cpuset_cpus": "2", "label": "mem"},

    #     # With stream allocation
    #     # {"client": client, "image": "stress-ng-mem-stream:latest", "command": ["stress-ng", "--stream", "4", "--stream-ops", "5000", "--metrics-brief"], "cpuset_cpus": "2", "label": "mem"},

    #     {"client": client, "image": "fio:latest", "command": ["fio", "--name", "seqread", "--rw", "read", "--bs", "1M", "--size", "18G", "--numjobs", "1", "--group_reporting"], "cpuset_cpus": "4", "label": "fileio"}

    # ]

    colocation = [
    {"CpuMem": [
        {"client": client, "image": "stress-ng-cpu:latest", "command": ["stress-ng", "--cpu", "1", "--cpu-method", "matrixprod", "--cpu-ops", "100000", "--metrics-brief"], "cpuset_cpus": "0", "label": "cpu"},
        {"client": client, "image": "stress-ng-mem-vm:latest", "command": ["stress-ng", "--vm", "1", "--vm-bytes", "18G", "--vm-ops", "1000", "--metrics-brief"], "cpuset_cpus": "24", "label": "mem"}
    ]},
    {"MemFileIO": [
        {"client": client, "image": "stress-ng-mem-vm:latest", "command": ["stress-ng", "--vm", "1", "--vm-bytes", "18G", "--vm-ops", "1000", "--metrics-brief"], "cpuset_cpus": "1", "label": "mem"},
        {"client": client, "image": "fio:latest", "command": ["fio", "--name", "seqread", "--rw", "read", "--bs", "1M", "--size", "18G", "--numjobs", "1", "--group_reporting"], "cpuset_cpus": "25", "label": "fileio"}
    ]},
    {"FileIOCpu": [
        {"client": client, "image": "fio:latest", "command": ["fio", "--name", "seqread", "--rw", "read", "--bs", "1M", "--size", "18G", "--numjobs", "1", "--group_reporting"], "cpuset_cpus": "2", "label": "fileio"},
        {"client": client, "image": "stress-ng-cpu:latest", "command": ["stress-ng", "--cpu", "1", "--cpu-method", "matrixprod", "--cpu-ops", "100000", "--metrics-brief"], "cpuset_cpus": "26", "label": "cpu"}
    ]},
    {"CpuCpu": [
        {"client": client, "image": "stress-ng-cpu:latest", "command": ["stress-ng", "--cpu", "1", "--cpu-method", "matrixprod", "--cpu-ops", "100000", "--metrics-brief"], "cpuset_cpus": "3", "label": "cpu"},
        {"client": client, "image": "stress-ng-cpu:latest", "command": ["stress-ng", "--cpu", "1", "--cpu-method", "matrixprod", "--cpu-ops", "100000", "--metrics-brief"], "cpuset_cpus": "27", "label": "cpu"}
    ]},
    {"MemMem": [
        {"client": client, "image": "stress-ng-mem-vm:latest", "command": ["stress-ng", "--vm", "1", "--vm-bytes", "18G", "--vm-ops", "1000", "--metrics-brief"], "cpuset_cpus": "4", "label": "mem"},
        {"client": client, "image": "stress-ng-mem-vm:latest", "command": ["stress-ng", "--vm", "1", "--vm-bytes", "18G", "--vm-ops", "1000", "--metrics-brief"], "cpuset_cpus": "28", "label": "mem"}
    ]},
    {"FileIOFileIO": [
        {"client": client, "image": "fio:latest", "command": ["fio", "--name", "seqread", "--rw", "read", "--bs", "1M", "--size", "18G", "--numjobs", "1", "--group_reporting"], "cpuset_cpus": "5", "label": "fileio"},
        {"client": client, "image": "fio:latest", "command": ["fio", "--name", "seqread", "--rw", "read", "--bs", "1M", "--size", "18G", "--numjobs", "1", "--group_reporting"], "cpuset_cpus": "29", "label": "fileio"}
    ]}
]

    
    for coloc in colocation:
            for pair_name, tasks in coloc.items():
                print(f"Running colocated tasks for: {pair_name}")
                with concurrent.futures.ThreadPoolExecutor(max_workers=2, thread_name_prefix="Colocator") as executor:
                    futures = [executor.submit(run_container, task) for task in tasks]
                for future in futures:
                    future.result()
                print(f"Completed colocated tasks for: {pair_name}")
    
    print("All colocated benchmarks completed.")
    # Clean up Docker client
    client.close() 
    
print("Container names and IDs:")
for name, container_id in coloc_benchmarking_results.items():
    print(f"{name}: {container_id}")

Running colocated tasks for: CpuMem
Container lifetime: 8.759432 seconds
Container on CPU 2 completed and removed.
Container lifetime: 103.116869 seconds
Container on CPU 0 completed and removed.
Completed colocated tasks for: CpuMem
Running colocated tasks for: MemFileIO
Container lifetime: 8.77126 seconds
Container on CPU 2 completed and removed.
Container lifetime: 30.257937 seconds
Container on CPU 4 completed and removed.
Completed colocated tasks for: MemFileIO
Running colocated tasks for: FileIOCpu
Container lifetime: 12.925777 seconds
Container on CPU 4 completed and removed.
Container lifetime: 103.534049 seconds
Container on CPU 0 completed and removed.
Completed colocated tasks for: FileIOCpu
Running colocated tasks for: CpuCpu
Container lifetime: 154.577175 seconds
Container on CPU 0 completed and removed.
Container lifetime: 206.207401 seconds
Container on CPU 0 completed and removed.
Completed colocated tasks for: CpuCpu
Running colocated tasks for: MemMem
Container lifet

In [ ]:
for name, container_id in isolated_benchmarking_results.items():
    print(f"{name}: {container_id}")

ecstatic_pasteur: {'coloc_pair': None, 'workload': 'mem', 'id': 'fe52bf17554b695520f18f38355a46d41e73036c36cfdcc5742c4ba54b191aeb', 'life_time': 7.257822}
crazy_nightingale: {'coloc_pair': None, 'workload': 'cpu', 'id': '66f0eac1530d6f8c56f4bb9946af437cc72af1bf1e08423398c6c121aee279c6', 'life_time': 38.029981}
trusting_sammet: {'coloc_pair': None, 'workload': 'fileio', 'id': '63db1a729f2eacde7c7d1c80da650bc885892c8aba655a13daefc2956a707bc3', 'life_time': 124.872952}


In [9]:
# Access the watched benchmark containers for runtime and power consumption.
coloc_benchmarking_results_copy = copy.deepcopy(coloc_benchmarking_results)

print("Container names and IDs:")
for name, container_id in coloc_benchmarking_results.items():
    print(f"{name}: {container_id}")
# print(benchmarking_results)

Container names and IDs:
interesting_snyder: {'coloc_pair': 'CpuMem', 'workload': 'mem', 'id': 'c619c6331745c4e2a5c3630a3edea55d9187fa641b5c5b34deec59e2f012a13e', 'colocated_runtime': 8.759432}
fervent_meninsky: {'coloc_pair': 'CpuMem', 'workload': 'cpu', 'id': '89e057c420ab19bc35e29158a5a304679586e46ef017cad06fc0429a0561a367', 'colocated_runtime': 103.116869}
brave_ride: {'coloc_pair': 'MemFileIO', 'workload': 'mem', 'id': '5e83719eb0998d51f65169d0c92928e2f163fce302f0eda9c56e8a1e660b5477', 'colocated_runtime': 8.77126}
great_murdock: {'coloc_pair': 'MemFileIO', 'workload': 'fileio', 'id': 'c3b19e5c424916a2f23974344444cb0569177011b6c391d2c799d76a96ae258f', 'colocated_runtime': 30.257937}
trusting_kowalevski: {'coloc_pair': 'FileIOCpu', 'workload': 'fileio', 'id': 'b96d844642120d0516f0806bcfabf05e98f4d2f815c3b382d5af7919bb60ee23', 'colocated_runtime': 12.925777}
clever_chatterjee: {'coloc_pair': 'FileIOCpu', 'workload': 'cpu', 'id': 'e203bfac3b61e610f0d88c793777e9192358d0b13934bd85240bd

In [16]:
# Access the watched benchmark containers for runtime and power consumption.
# Add power consumption caputured by ebpf-mon.
output_dir = "output"


print("Container names and IDs:")
for name, container_id in coloc_benchmarking_results.items():
    print(f"{name}: {container_id}")

# Collect container IDs
container_ids = set(info['id'] for info in coloc_benchmarking_results.values())

for entry in os.listdir(output_dir):
    entry_path = os.path.join(output_dir, entry)
    if os.path.isdir(entry_path):
        # If entry is NOT part of any container ID, remove the whole directory and continue
        if not any(entry in cid for cid in container_ids):
            # print(f"Removing directory: {entry_path}")
            shutil.rmtree(entry_path)
            continue  

        # If entry IS part of a container ID, remove all subfolders/files not starting with "power"
        for metric_entry in os.listdir(entry_path):
            metric_entry_path = os.path.join(entry_path, metric_entry)
            if not metric_entry.startswith("power"):
                if os.path.isdir(metric_entry_path):
                    # print(f"Removing directory: {metric_entry_path}")
                    shutil.rmtree(metric_entry_path)
                else:
                    # print(f"Removing file: {metric_entry_path}")
                    os.remove(metric_entry_path)
            else:
                # If the entry is a power metric, keep it
                print(f"Keeping power metric: {metric_entry_path}")
                for file in os.listdir(metric_entry_path):
                    file_path = os.path.join(metric_entry_path, file)
                    print(f"Keeping file: {file_path}")
                    df = pd.read_csv(file_path)
                    container_name = df['container_name'].iloc[0] if 'container_name' in df.columns else None
                    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
                    df.set_index('timestamp', inplace=True)
                    power_mean = df['value'].mean()
                    print(f"Power mean for {file_path}: {power_mean}")
                    if container_name and container_name in coloc_benchmarking_results:
                        coloc_benchmarking_results[container_name]['power'] = float(power_mean)
# Print the final isolated benchmarking results with power consumption
print("Final colocated benchmarking results with power consumption:")
pprint.pprint(coloc_benchmarking_results)

Container names and IDs:
silly_panini: {'coloc_pair': 'CpuMem', 'workload': 'mem', 'id': '2937bf61724c94c1d80cd17967f2bda5215784e1f59ed8dc1e76ed2d07e494ca', 'colocated_runtime': 15.532565, 'isolated_runtime': 15.550664, 'isolated_power_consumption': 0.15277761160110087}
unruffled_shannon: {'coloc_pair': 'CpuMem', 'workload': 'cpu', 'id': '10a1eec13591cd80efd118904b166ae966c51d8efc11cb5a2909093c51cf9bae', 'colocated_runtime': 37.982324, 'isolated_runtime': 39.575586, 'isolated_power_consumption': 0.36692103170365514}
amazing_clarke: {'coloc_pair': 'MemFileIO', 'workload': 'mem', 'id': '6ac65f625ff79274017e36e5d4438acc0b3e7fe1c1141a11b8f8012e552a55f7', 'colocated_runtime': 15.584474, 'isolated_runtime': 15.550664, 'isolated_power_consumption': 0.15277761160110087}
intelligent_poitras: {'coloc_pair': 'MemFileIO', 'workload': 'fileio', 'id': '3ea6709968a709ed5dfb9c5ce1f0eae2c2f2da8b62b057b49996797cc3892f61', 'colocated_runtime': 181.926022, 'isolated_runtime': 182.115732, 'isolated_power_c

In [10]:
def calc_average_slowdown(slowdown_1, slowdown_2):
    average_slowdown = (slowdown_1 + slowdown_2) / 2
    return average_slowdown

def calc_slowdown_factor(isolated_runtime_1, isolated_runtime_2, coloc_runtime_1, coloc_runtime_2, isolated_power_consumption_1, isolated_power_consumption_2, coloc_power_consumption_1, coloc_power_consumption_2):
    slowdown_1 = isolated_runtime_1 / coloc_runtime_1
    slowdown_2 = isolated_runtime_2 / coloc_runtime_2
    power_slowdown_1 = isolated_power_consumption_1 / coloc_power_consumption_1
    power_slowdown_2 = isolated_power_consumption_2 / coloc_power_consumption_2
    average_runtime_slowdown = calc_average_slowdown(slowdown_1, slowdown_2)
    average_power_slowdown = calc_average_slowdown(power_slowdown_1, power_slowdown_2) 
    average_slowdown = (average_runtime_slowdown + average_power_slowdown) / 2

    return average_slowdown, 

# def calc_affinity_score(isolated_runtime_1, coloc_runtime_1, isolated_runtime_2, coloc_runtime_2):
#     affinity_score = (isolated_runtime_1 + isolated_runtime_2) / (coloc_runtime_1 + coloc_runtime_2)
#     return min(1, affinity_score)

def calc_affinity_score(isolated_runtime_1, coloc_runtime_1, isolated_runtime_2, coloc_runtime_2,
                        isolated_power_consumption_1, coloc_power_consumption_1,
                        isolated_power_consumption_2, coloc_power_consumption_2):
    # Runtime affinity
    runtime_affinity = (isolated_runtime_1 + isolated_runtime_2) / (coloc_runtime_1 + coloc_runtime_2)
    # Power affinity
    power_affinity = (isolated_power_consumption_1 + isolated_power_consumption_2) / (coloc_power_consumption_1 + coloc_power_consumption_2)
    # Average both
    affinity_score = (runtime_affinity + power_affinity) / 2
    # return min(1, affinity_score)
    return  affinity_score
    
for name, result in coloc_benchmarking_results.items():
    coloc_workload = result.get('workload')
    # Find the isolated runtime for the same workload
    isolated_runtime = ''
    isolated_power_consumption = ''
    for iso_name, iso_result in isolated_benchmarking_results.items():
        if iso_result.get('workload') == coloc_workload and iso_result.get('coloc_pair') is None:
            isolated_runtime = iso_result.get('life_time')
            isolated_power_consumption = iso_result.get('power')
            break
    result['isolated_runtime'] = isolated_runtime
    result['isolated_power_consumption'] = isolated_power_consumption
    print(f"Colocated benchmark: {coloc_workload}, {result.get('coloc_pair')}, Isolated runtime: {isolated_runtime}, Isolated power consumption: {isolated_power_consumption}")

Colocated benchmark: mem, CpuMem, Isolated runtime: 8.778812, Isolated power consumption: None
Colocated benchmark: cpu, CpuMem, Isolated runtime: 104.159797, Isolated power consumption: None
Colocated benchmark: mem, MemFileIO, Isolated runtime: 8.778812, Isolated power consumption: None
Colocated benchmark: fileio, MemFileIO, Isolated runtime: 30.129789, Isolated power consumption: None
Colocated benchmark: fileio, FileIOCpu, Isolated runtime: 30.129789, Isolated power consumption: None
Colocated benchmark: cpu, FileIOCpu, Isolated runtime: 104.159797, Isolated power consumption: None
Colocated benchmark: cpu, CpuCpu, Isolated runtime: 104.159797, Isolated power consumption: None
Colocated benchmark: cpu, CpuCpu, Isolated runtime: 104.159797, Isolated power consumption: None
Colocated benchmark: mem, MemMem, Isolated runtime: 8.778812, Isolated power consumption: None
Colocated benchmark: mem, MemMem, Isolated runtime: 8.778812, Isolated power consumption: None
Colocated benchmark: f

In [11]:
# Transform benchmarking dict into a summary
coloc_summary = defaultdict(dict)

for name, result in coloc_benchmarking_results.items():
    pair = result['coloc_pair']
    workload = result.get('workload', 'unknown')
    if 'workload_1' not in coloc_summary[pair]:
        coloc_summary[pair]['workload_1'] = workload
        coloc_summary[pair]['colocated_runtime_1'] = result.get('colocated_runtime')  
        coloc_summary[pair]['isolated_runtime_1'] = result.get('isolated_runtime')
        coloc_summary[pair]['isolated_power_consumption_1'] = result.get('isolated_power_consumption')
        coloc_summary[pair]['colocated_power_consumption_1'] = result.get('power')
        
    else:
        coloc_summary[pair]['workload_2'] = workload
        coloc_summary[pair]['colocated_runtime_2'] = result.get('colocated_runtime')  
        coloc_summary[pair]['isolated_runtime_2'] = result.get('isolated_runtime')
        coloc_summary[pair]['isolated_power_consumption_2'] = result.get('isolated_power_consumption')
        coloc_summary[pair]['colocated_power_consumption_2'] = result.get('power')
        
# calculate and add average_slowdown and affinity_score for each coloc pair
for pair, summary in coloc_summary.items():
    try:
        iso_run1 = float(summary['isolated_runtime_1'])
        iso_run2 = float(summary['isolated_runtime_2'])
        coloc_run1 = float(summary['colocated_runtime_1'])
        coloc_run2 = float(summary['colocated_runtime_2'])
        iso_pow1 = float(summary['isolated_power_consumption_1'])
        iso_pow2 = float(summary['isolated_power_consumption_2'])
        coloc_pow1 = float(summary['colocated_power_consumption_1'])
        coloc_pow2 = float(summary['colocated_power_consumption_2'])
        summary['average_slowdown'] = calc_slowdown_factor(iso_run1, iso_run2, coloc_run1, coloc_run2, iso_pow1, iso_pow2, coloc_pow1, coloc_pow2)
        # summary['affinity_score'] = calc_affinity_score(iso_run1, coloc_run1, iso_run2, coloc_run2, iso_pow1, coloc_pow1, iso_pow2,coloc_pow2)
        summary['affinity_score'] = calc_affinity_score(
    iso_run1, coloc_run1, iso_run2, coloc_run2,
    iso_pow1, coloc_pow1, iso_pow2, coloc_pow2
)
    except Exception as e:
        summary['average_slowdown'] = None
        summary['affinity_score'] = None
        print(f"Could not calculate for pair {pair}: {e}")

coloc_summary = dict(coloc_summary)
pprint.pprint(coloc_summary)

Could not calculate for pair CpuMem: float() argument must be a string or a real number, not 'NoneType'
Could not calculate for pair MemFileIO: float() argument must be a string or a real number, not 'NoneType'
Could not calculate for pair FileIOCpu: float() argument must be a string or a real number, not 'NoneType'
Could not calculate for pair CpuCpu: float() argument must be a string or a real number, not 'NoneType'
Could not calculate for pair MemMem: float() argument must be a string or a real number, not 'NoneType'
Could not calculate for pair FileIOFileIO: float() argument must be a string or a real number, not 'NoneType'
{'CpuCpu': {'affinity_score': None,
            'average_slowdown': None,
            'colocated_power_consumption_1': None,
            'colocated_power_consumption_2': None,
            'colocated_runtime_1': 154.577175,
            'colocated_runtime_2': 206.207401,
            'isolated_power_consumption_1': None,
            'isolated_power_consumption_2': 

In [20]:
# Write affinity score matrix to a CSV file
affinity_score_matrix = pd.DataFrame.from_dict(coloc_summary, orient='index')
affinity_score_matrix = affinity_score_matrix[['workload_1', 'workload_2', 'affinity_score']]

print(affinity_score_matrix)
# Write back to csv
affinity_score_matrix.to_csv('affinity_score_matrix.csv', index=False)

             workload_1 workload_2  affinity_score
CpuMem              mem        cpu        0.946344
MemFileIO           mem     fileio       24.642740
FileIOCpu           cpu     fileio        0.982372
CpuCpu              cpu        cpu        0.604017
MemMem              mem        mem        9.789625
FileIOFileIO     fileio     fileio       17.827398


### Host-level co-location & energy-efficiency experiments

#### (1) stress-ng CPU benchmark pinned to 20%, 50% & 100% of available cores. GUDE EPC 8045-1 records energy

In [ ]:
# stress-ng
def parse_start_time(start_time_str):
    if '.' in start_time_str:
        time_part, rest = start_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(start_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def parse_die_time(die_time_str):
    if '.' in die_time_str:
        time_part, rest = die_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(die_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def run_container(task):
    label = task.get("label")
    run_kwargs = {}
    if "environment" in task:
        run_kwargs["environment"] = task["environment"]
    if "volumes" in task:
        run_kwargs["volumes"] = task["volumes"]
    if "working_dir" in task:
        run_kwargs["working_dir"] = task["working_dir"]
    container = task['client'].containers.run(
        image=task['image'],
        command=task['command'],
        cpuset_cpus=task.get('cpuset_cpus'),
        cgroupns="private",
        detach=True,
        labels = {"test": label} if label else {},
        **run_kwargs
    )
    container.wait()

    # fetch benchmark output
    logs = container.logs(stdout=True, stderr=True).decode("utf-8", errors="replace")
    
    container.reload()
    status = container.attrs['State']['ExitCode']
    if status != 0:
        print(container.logs().decode())
    die_time = parse_die_time(container.attrs['State']['FinishedAt'])
    started_at = container.attrs['State']['StartedAt']
    start_time = parse_start_time(started_at)
    container_lifetime = (die_time - start_time).total_seconds()
    print(f"Container lifetime: {container_lifetime} seconds")
    print(f"Container on CPU {task.get('cpuset_cpus')} completed and removed.")
    isolated_benchmarking_results[container.name] = {
        # 'workload': container.attrs['Config']['Labels'],
        'coloc_pair': None,
        'workload': container.attrs['Config']['Labels'].get('test', None), 
        'id': container.id,
        'life_time': container_lifetime,
    }
    container.remove()

    print(logs)
    

if __name__ == "__main__":
    client = docker.from_env()
    isolated_benchmarking_results = {}

    tasks = [
        {"client": client, "image": "stress-ng-cpu:latest", "command": ["stress-ng", "--cpu", "10", "--cpu-method", "matrixprod", "--timeout","120","--metrics-brief"], "cpuset_cpus": "0", "label": "cpu"},
        {"client": client, "image": "stress-ng-cpu:latest", "command": ["stress-ng", "--cpu", "24", "--cpu-method", "matrixprod", "--timeout","120","--metrics-brief"], "cpuset_cpus": "0", "label": "cpu"},
        {"client": client, "image": "stress-ng-cpu:latest", "command": ["stress-ng", "--cpu", "48", "--cpu-method", "matrixprod", "--timeout","120","--metrics-brief"], "cpuset_cpus": "0", "label": "cpu"},
    ]

    for task in tasks:  
        run_container(task)

    print("All host-level benchmarks completed.")
    client.close()

Container lifetime: 122.305361 seconds
Container on CPU 0 completed and removed.
Container lifetime: 124.897762 seconds
Container on CPU 0 completed and removed.
Container lifetime: 129.175801 seconds
Container on CPU 0 completed and removed.
All host-level benchmarks completed.


In [18]:
for name, container_id in isolated_benchmarking_results.items():
    print(f"{name}: {container_id}")

funny_chatterjee: {'coloc_pair': None, 'workload': 'cpu', 'id': '95f96829b877cc9076cb67f3386a58a9072907c18b9537b2ed58afd5602cf86e', 'life_time': 122.305361}
elegant_rosalind: {'coloc_pair': None, 'workload': 'cpu', 'id': '6dd447c4438e0c5799212171e9061a67c3cd2e6d5d44c36d8e7f9d1d9c537943', 'life_time': 124.897762}
elegant_spence: {'coloc_pair': None, 'workload': 'cpu', 'id': '75372bf86b3798fa15ec2803a93a9aa1693449c877d27e29de0c6698acd0043d', 'life_time': 129.175801}


#### (2) stress-ng + fio CPU, Memory & File I/0 benchmarks are pinned to 50% of available cores. GUDE EPC 8045-1 records energy

In [15]:
# stress-ng + fio
def parse_start_time(start_time_str):
    if '.' in start_time_str:
        time_part, rest = start_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(start_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def parse_die_time(die_time_str):
    if '.' in die_time_str:
        time_part, rest = die_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(die_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def run_container(task):
    label = task.get("label")
    run_kwargs = {}
    if "environment" in task:
        run_kwargs["environment"] = task["environment"]
    if "volumes" in task:
        run_kwargs["volumes"] = task["volumes"]
    if "working_dir" in task:
        run_kwargs["working_dir"] = task["working_dir"]
    container = task['client'].containers.run(
        image=task['image'],
        command=task['command'],
        cpuset_cpus=task.get('cpuset_cpus'),
        cgroupns="private",
        detach=True,
        labels = {"test": label} if label else {},
        **run_kwargs
    )
    container.wait()
    
    # fetch benchmark output
    logs = container.logs(stdout=True, stderr=True).decode("utf-8", errors="replace")
    
    container.reload()
    status = container.attrs['State']['ExitCode']
    if status != 0:
        print(container.logs().decode())
    die_time = parse_die_time(container.attrs['State']['FinishedAt'])
    started_at = container.attrs['State']['StartedAt']
    start_time = parse_start_time(started_at)
    container_lifetime = (die_time - start_time).total_seconds()
    print(f"Container lifetime: {container_lifetime} seconds")
    print(f"Container on CPU {task.get('cpuset_cpus')} completed and removed.")
    isolated_benchmarking_results[container.name] = {
        # 'workload': container.attrs['Config']['Labels'],
        'coloc_pair': None,
        'workload': container.attrs['Config']['Labels'].get('test', None), 
        'id': container.id,
        'life_time': container_lifetime,
    }
    container.remove()
    
    print(logs)

if __name__ == "__main__":
    client = docker.from_env()
    isolated_benchmarking_results = {}

    tasks = [
        {"client": client, "image": "stress-ng-cpu:latest", "command": ["stress-ng", "--cpu", "24", "--cpu-method", "matrixprod", "--timeout","120","--metrics-brief"], "cpuset_cpus": "0", "label": "cpu"},
        {"client": client, "image": "stress-ng-mem-vm:latest", "command": ["stress-ng", "--vm", "24", "--vm-bytes", "1G", "--timeout", "120", "--metrics-brief"], "cpuset_cpus": "24-47", "label": "mem"},
        {"client": client, "image": "fio:latest", "command": ["fio", "--name", "seqread", "--rw", "read", "--bs", "4K", "--size", "768M","--ioengine","io_uring","--runtime","120","--iodepth","16","--direct","1", "--numjobs", "24", "--group_reporting"], "cpuset_cpus": "24-47", "label": "fileio"}
    ]

    for task in tasks:  
        run_container(task)

    print("All host-level benchmarks completed.")
    client.close()

Container lifetime: 121.360108 seconds
Container on CPU 0 completed and removed.
stress-ng: info:  [1] setting to a 2 mins, 0 secs run per stressor
stress-ng: info:  [1] dispatching hogs: 24 cpu
stress-ng: info:  [1] note: /proc/sys/kernel/sched_autogroup_enabled is 1 and this can impact scheduling throughput for processes not attached to a tty. Setting this to 0 may improve performance metrics
stress-ng: metrc: [1] stressor       bogo ops real time  usr time  sys time   bogo ops/s     bogo ops/s
stress-ng: metrc: [1]                           (secs)    (secs)    (secs)   (real time) (usr+sys time)
stress-ng: metrc: [1] cpu               70808    120.09    120.24      0.00       589.62         588.87
stress-ng: info:  [1] skipped: 0
stress-ng: info:  [1] passed: 24: cpu (24)
stress-ng: info:  [1] failed: 0
stress-ng: info:  [1] metrics untrustworthy: 0
stress-ng: info:  [1] successful run completed in 2 mins, 0.28 secs

Container lifetime: 120.40112 seconds
Container on CPU 24-47 compl

#### (3) stress-ng + fio CPU, Memory & File I/O benchmarks are co-located on a host with equal core sharing of 50% of available cores. GUDE EPC 8045-1 records energy

In [ ]:
# Using stress-ng and fio
def parse_start_time(start_time_str):
    # Trim to microseconds and remove trailing 'Z'
    if '.' in start_time_str:
        time_part, rest = start_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(start_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")

def parse_die_time(die_time_str):
    if '.' in die_time_str:
        time_part, rest = die_time_str.split('.')
        microseconds = rest[:6]  
        return datetime.strptime(f"{time_part}.{microseconds}", "%Y-%m-%dT%H:%M:%S.%f")
    return datetime.strptime(die_time_str.replace('Z', ''), "%Y-%m-%dT%H:%M:%S")
    
def run_container(task):
    label = task.get("label")
    run_kwargs = {}
    if "environment" in task:
        run_kwargs["environment"] = task["environment"]
    if "volumes" in task:
        run_kwargs["volumes"] = task["volumes"]
    if "working_dir" in task:
        run_kwargs["working_dir"] = task["working_dir"]
    container = task['client'].containers.run(
        image=task['image'],
        privileged=True,
        command=task['command'],
        cpuset_cpus=task.get('cpuset_cpus'),
        cgroupns="private",
        detach=True,
        labels = {"test": label} if label else {},
        **run_kwargs
    )
    
    container.wait()

    # fetch benchmark output
    logs = container.logs(stdout=True, stderr=True).decode("utf-8", errors="replace")
    
    container.reload()
    status = container.attrs['State']['ExitCode']
    if status != 0:
        print(container.logs().decode())
    die_time = parse_die_time(container.attrs['State']['FinishedAt'])
    started_at = container.attrs['State']['StartedAt']
    start_time = parse_start_time(started_at)
    container_lifetime = (die_time - start_time).total_seconds()
    print(f"Container lifetime: {container_lifetime} seconds")
    coloc_benchmarking_results[container.name] = {
        'coloc_pair': pair_name,
        'workload': container.attrs['Config']['Labels'].get('test', None),  
        'id': container.id,
        'colocated_runtime': container_lifetime,
    }
    container.remove()
    print(f"Container on CPU {task.get('cpuset_cpus')} completed and removed.")
    
    print(logs)

    
if __name__ == "__main__":
    
    # Initialize Docker client
    client = docker.from_env()
    
    # Prepare affinity score map for colocated pairs.
    coloc_benchmarking_results = {}

    colocation = [
    {"CpuMem": [
        {"client": client, "image": "stress-ng-cpu:latest", "command": ["stress-ng", "--cpu", "24", "--cpu-method", "matrixprod", "--timeout","120", "--metrics-brief"], "cpuset_cpus": "0-23", "label": "cpu"},
        {"client": client, "image": "stress-ng-mem-vm:latest", "command": ["stress-ng", "--vm", "24", "--vm-bytes", "1G", "--timeout","120", "--metrics-brief"], "cpuset_cpus": "24-47", "label": "mem"}
    ]},
    {"MemFileIO": [
        {"client": client, "image": "stress-ng-mem-vm:latest", "command": ["stress-ng", "--vm", "24", "--vm-bytes", "1G", "--timeout","120","--metrics-brief"], "cpuset_cpus": "0-23", "label": "mem"},
        {"client": client, "image": "fio:latest", "command": ["fio", "--name", "seqread", "--rw", "read", "--bs", "4K", "--size", "768M","--ioengine","io_uring","--iodepth","16","--direct","1", "--numjobs", "24", "--group_reporting"], "cpuset_cpus": "24-47", "label": "fileio"},
    ]},
    {"FileIOCpu": [
        {"client": client, "image": "fio:latest", "command": ["fio", "--name", "seqread", "--runtime","120","--rw", "read", "--bs", "4K", "--size", "768M","--ioengine","io_uring","--iodepth","16","--direct","1", "--numjobs", "24", "--group_reporting"], "cpuset_cpus": "0-23", "label": "fileio"},
        {"client": client, "image": "stress-ng-cpu:latest", "command": ["stress-ng", "--cpu", "24", "--cpu-method", "matrixprod","--timeout","120", "--metrics-brief"], "cpuset_cpus": "24-47", "label": "cpu"}
    ]}
]

    
    for coloc in colocation:
            for pair_name, tasks in coloc.items():
                print(f"Running colocated tasks for: {pair_name}")
                with concurrent.futures.ThreadPoolExecutor(max_workers=2, thread_name_prefix="Colocator") as executor:
                    futures = [executor.submit(run_container, task) for task in tasks]
                for future in futures:
                    future.result()
                print(f"Completed colocated tasks for: {pair_name}")
    
    print("All colocated benchmarks completed.")
    # Clean up Docker client
    client.close() 
    
print("Container names and IDs:")
for name, container_id in coloc_benchmarking_results.items():
    print(f"{name}: {container_id}")

Running colocated tasks for: CpuMem
Container lifetime: 12.514842 seconds
Container on CPU 24-47 completed and removed.
stress-ng: info:  [1] setting to a 12 secs run per stressor
stress-ng: info:  [1] dispatching hogs: 24 vm
stress-ng: info:  [1] note: /proc/sys/kernel/sched_autogroup_enabled is 1 and this can impact scheduling throughput for processes not attached to a tty. Setting this to 0 may improve performance metrics
stress-ng: metrc: [1] stressor       bogo ops real time  usr time  sys time   bogo ops/s     bogo ops/s
stress-ng: metrc: [1]                           (secs)    (secs)    (secs)   (real time) (usr+sys time)
stress-ng: metrc: [1] vm              4544482     12.09     44.78     13.76    375801.18       77642.82
stress-ng: info:  [1] skipped: 0
stress-ng: info:  [1] passed: 24: vm (24)
stress-ng: info:  [1] failed: 0
stress-ng: info:  [1] metrics untrustworthy: 0
stress-ng: info:  [1] successful run completed in 12.21 secs

Completed colocated tasks for: CpuMem
Runni

KeyboardInterrupt: 

Container lifetime: 81.426455 seconds
Container on CPU 24-47 completed and removed.
seqread: (g=0): rw=read, bs=(R) 4096B-4096B, (W) 4096B-4096B, (T) 4096B-4096B, ioengine=io_uring, iodepth=16
...
fio-3.36
Starting 24 processes
seqread: Laying out IO file (1 file / 768MiB)
seqread: Laying out IO file (1 file / 768MiB)
seqread: Laying out IO file (1 file / 768MiB)
seqread: Laying out IO file (1 file / 768MiB)
seqread: Laying out IO file (1 file / 768MiB)
seqread: Laying out IO file (1 file / 768MiB)
seqread: Laying out IO file (1 file / 768MiB)
seqread: Laying out IO file (1 file / 768MiB)
seqread: Laying out IO file (1 file / 768MiB)
seqread: Laying out IO file (1 file / 768MiB)
seqread: Laying out IO file (1 file / 768MiB)
seqread: Laying out IO file (1 file / 768MiB)
seqread: Laying out IO file (1 file / 768MiB)
seqread: Laying out IO file (1 file / 768MiB)
seqread: Laying out IO file (1 file / 768MiB)
seqread: Laying out IO file (1 file / 768MiB)
seqread: Laying out IO file (1 file 

In [ ]:
# TODO: Calculate energy efficiency
# Repeat for CPU, Mem and FileIO benchmarks
# Output: bogo-ops/sec
# Gude gives P(t=120) in Watts = (Joules / seconds)
# Mean Power during workload 
# Mean Power during idle baseline
# Dynamic Power = Mean Power during workload - Mean Power during idle baseline
# Dynamic Energy = Dynamic Power * Runtime (seconds)
# Total Work = W = R x T
# Energy Efficiency = W / E = (R x T) / (P x T) = R / P